# Class for accessing Steam game data

In [90]:
import requests
from bs4 import BeautifulSoup
import re

class Steam:
    def __init__(self):
        self.session = requests.Session()
    
    def get_game_html(self, url):
        try:
            r = self.session.get(url)
            r.raise_for_status()
            return r.content
        except requests.exceptions.HTTPError as err:
            print('error: ', err)
    
    def strip_features(self, html):
        game = {}
        keys = ['SteamURL', 'Genres', 'Indie', 'Soundtrack', 'Franchise', 
                'OriginalCost', 'DiscountedCost', 'Players', 'Controller', 
                'Languages', 'ESRB', 'Tags']

        values = []

        soup = BeautifulSoup(html)

        # steam_url
        steam_url = soup.find(
            'div', class_='breadcrumbs').find_all('a')[-1].get('href')
        values.append(steam_url)

        # contains several details we want
        details = soup.find('div', class_='details_block').get_text()

        # genres
        genres = self.__re_detail('genre', details)
        values.append(genres)
        
        # list of tags for extracting features
        tags = [ tag.get_text().strip() for tag in soup.find_all('a', class_='app_tag') ]

        # indie and soundtrack
        for tag in ['Indie', 'Soundtrack']:
            if tag in tags:
                tags.remove(tag)
                values.append(True)
            else:
                values.append(False)

        # franchise
        franchise = self.__re_detail('franchise', details)
        values.append(franchise)

        # original_cost
        original_cost = 

        # add the rest of the tags as seperate feature
        values.append(', '.join(tags))


        game = zip(keys, values)
        return game

    def __re_detail(self, detail, text):
        return re.search(f'{detail.title()}:[\s\n].*', text).group().split(':')[1].strip()
  

## Test

In [91]:
# instantiate
s = Steam()

# get game information
html = s.get_game_html('https://store.steampowered.com/app/1118010/Monster_Hunter_World_Iceborne/')
print(html[:100])

# pull features from html
features = s.strip_features(html)
for key, val in features:
    print(f'{key}: {val}')

b'<!DOCTYPE html>\r\n<html class=" responsive" lang="en">\r\n<head>\r\n\t<meta http-equiv="Content-Type" cont'
SteamURL: https://store.steampowered.com/app/1118010/?snr=1_5_9__205
Genres: Action
Indie: False
Soundtrack: False
Franchise: Monster Hunter
OriginalCost: Action, Multiplayer, Co-op, Hunting, Open World, Difficult, Masterpiece, Online Co-Op, Third Person, Singleplayer, Adventure, RPG, Character Customization, Exploration, Hack and Slash, JRPG, Story Rich, Anime, Souls-like
